In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import h5py
import healpy as hp
import kalepy as kale

import holodeck as holo
import holodeck.anisotropy as anis
from holodeck import detstats, plot, utils
from holodeck.constants import YR, MSOL, GYR

# Set Up

In [ ]:
dur, cad = 16.03*YR, 0.2*YR
fobs_gw_cents = utils.nyquist_freqs(dur,cad)
fobs_gw_edges = utils.nyquist_freqs_edges(dur,cad)
# sam = holo.sam.Semi_Analytic_Model()
sam = holo.sam.Semi_Analytic_Model(shape=20)  # faster version


In [ ]:
fobs_orb_cents = fobs_gw_cents/2.0
fobs_orb_edges = fobs_gw_edges/2.0
# hard = holo.hardening.Hard_GW()
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
redz_final, diff_num = holo.sam_cython.dynamic_binary_number_at_fobs(
    fobs_orb_cents, sam, hard, holo.cosmo)
edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]

In [ ]:
if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
    hard_name = 'Fixed Time'
elif isinstance(hard, holo.hardening.Hard_GW):
    hard_name = 'GW Only'

# Calculate Anisotropy

$$ C_\ell (f) = \delta_{\ell 0}\delta_{m0} \bigg( \frac{f}{4\pi \Delta f}   \int d \vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^2 (f,\vec{\theta})   \bigg)^2 
+ \big( \frac{f}{4 \pi \Delta f}\big)^2 \int d\vec{\theta} \frac{d N_{\Delta f}}{d \vec{\theta}} h^4 (f, \vec{\theta})
$$



* dens = d^3 n / [dlog10M dq dz] in units of [Mpc^-3] 
* dnum = d^4N / dlog10M dq dz dlnf
* number = dN /dlnf

 ## Cl_analytic_from_num()

In [ ]:
hs = holo.gravwaves.strain_amp_from_bin_edges_redz(edges, redz_final)
cynum = holo.sam_cython.integrate_differential_number_3dx1d(edges, diff_num)
C0_cynum, Cl_cynum = anis.Cl_analytic_from_num(fobs_orb_edges, number=cynum, hs=hs)
C0_cyreals, Cl_cyreals = anis.Cl_analytic_from_num(fobs_orb_edges, cynum, hs, realize=20)

utnum = utils._integrate_grid_differential_number(edges, diff_num, freq=False)
utnum = utnum * np.diff(np.log(fobs_gw_edges))
C0_utnum, Cl_utnum = anis.Cl_analytic_from_num(fobs_orb_edges, number=utnum, hs=hs)
C0_utreals, Cl_utreals = anis.Cl_analytic_from_num(fobs_orb_edges, utnum, hs, realize=20)



In [ ]:
fig = anis.plot_ClC0_versions(fobs_gw_cents)
ax = fig.axes[0]

anis.draw_analytic(ax, Cl_cynum, C0_cynum, fobs_gw_cents, color='tab:orange', label='cython number', alpha=0.5, lw=4)
anis.draw_reals(ax, Cl_cyreals, C0_cyreals, fobs_gw_cents, color='tab:orange', label=None,
                show_reals=True, show_median=True, show_ci=True)

anis.draw_analytic(ax, Cl_utnum, C0_utnum, fobs_gw_cents, color='tab:red', label='utils number', alpha=0.5)
anis.draw_reals(ax, Cl_utreals, C0_utreals, fobs_gw_cents, color='tab:red', label=None,
                                show_reals=True, show_median=True, show_ci=True)

fig.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=3)

This confirms that using number function is same, regardless which number we use

## Cl_analytic_from_dnum

In [ ]:
C0_dnum, Cl_dnum = anis.Cl_analytic_from_dnum(edges, diff_num)
C0_dnum_reals, Cl_dnum_reals = anis.Cl_analytic_from_dnum(edges, diff_num, realize=10)
C0_redz, Cl_redz = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final)
C0_redz_reals, Cl_redz_reals = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final, realize=10)

In [ ]:
arr = np.array([1,2,3,4,])
print(arr[:,np.newaxis].shape)

In [ ]:
print(C0_dnum.shape)
print(C0_dnum_reals.shape)

note that Cl_best does not use the same model as the mockups for Sato-Polito method here!

In [ ]:
fig = anis.plot_ClC0_versions(fobs_gw_cents)
ax = fig.axes[0]

anis.draw_analytic(ax, Cl_cynum, C0_cynum, fobs_gw_cents, color='tab:orange', label='cython number', alpha=0.5, lw=4)
anis.draw_reals(ax, Cl_cyreals, C0_cyreals, fobs_gw_cents, color='tab:orange', label=None,
                show_reals=True, show_median=True, show_ci=True)

anis.draw_analytic(ax, Cl_dnum, C0_dnum, fobs_gw_cents, label='dnum, z_init', color='deeppink')
anis.draw_reals(ax, Cl_dnum_reals, C0_dnum_reals, fobs_gw_cents, label=None, color='deeppink')

anis.draw_analytic(ax, Cl_redz, C0_redz, fobs_gw_cents, label='dnum, z_final', color='indigo')
anis.draw_reals(ax, Cl_redz_reals, C0_redz_reals, fobs_gw_cents, label=None, color='indigo')

fig.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=4)
ax.set_title('Shape=%s, %s' % (str(sam.shape), str(hard_name)), fontsize=14)

fig.tight_layout()


# Compare Models

In [ ]:
def compare_all_analytic_anis(sam, hard, fobs_gw_cents, fobs_gw_edges):

    fobs_orb_cents = fobs_gw_cents/2.0
    fobs_orb_edges = fobs_gw_edges/2.0
    redz_final, diff_num = holo.sam_cython.dynamic_binary_number_at_fobs(
        fobs_orb_cents, sam, hard, holo.cosmo)
    edges = [sam.mtot, sam.mrat, sam.redz, fobs_orb_edges]
    if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
        hard_name = 'Fixed Time'
    elif isinstance(hard, holo.hardening.Hard_GW):
        hard_name = 'GW Only'

    # analytic from number
    print('calculating analytic from cython number')
    hs = holo.gravwaves.strain_amp_from_bin_edges_redz(edges, redz_final)
    cynum = holo.sam_cython.integrate_differential_number_3dx1d(edges, diff_num)
    C0_cynum, Cl_cynum = anis.Cl_analytic_from_num(fobs_orb_edges, number=cynum, hs=hs)
    C0_cyreals, Cl_cyreals = anis.Cl_analytic_from_num(fobs_orb_edges, cynum, hs, realize=20)

    # anayltic from cython number
    utnum = utils._integrate_grid_differential_number(edges, diff_num, freq=False)
    utnum = utnum * np.diff(np.log(fobs_gw_edges))
    C0_utnum, Cl_utnum = anis.Cl_analytic_from_num(fobs_orb_edges, number=utnum, hs=hs)
    C0_utreals, Cl_utreals = anis.Cl_analytic_from_num(fobs_orb_edges, utnum, hs, realize=20)

    # analytic from dnum
    print('calculating analytic from dnum, initial redshift')
    C0_dnum, Cl_dnum = anis.Cl_analytic_from_dnum(edges, diff_num)
    C0_dnum_reals, Cl_dnum_reals = anis.Cl_analytic_from_dnum(edges, diff_num, realize=10)

    print('calculating analytic from dnum, final redshift')
    C0_redz, Cl_redz = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final)
    C0_redz_reals, Cl_redz_reals = anis.Cl_analytic_from_dnum(edges, diff_num, redz_final, realize=10)

    # plot everything
    print('plotting')
    fig = anis.plot_ClC0_versions(fobs_gw_cents)
    ax = fig.axes[0]

    anis.draw_analytic(ax, Cl_cynum, C0_cynum, fobs_gw_cents, color='tab:orange', label='cython number', lw=4)
    anis.draw_reals(ax, Cl_cyreals, C0_cyreals, fobs_gw_cents, color='tab:orange', label=None,
                    show_reals=True, show_median=True, show_ci=True)

    anis.draw_analytic(ax, Cl_utnum, C0_utnum, fobs_gw_cents, color='tab:red', label='utils number', lw=3)
    anis.draw_reals(ax, Cl_utreals, C0_utreals, fobs_gw_cents, color='tab:red', label=None,
                    show_reals=True, show_median=True, show_ci=True)

    anis.draw_analytic(ax, Cl_dnum, C0_dnum, fobs_gw_cents, label='dnum, z_init', color='deeppink', lw=4)
    anis.draw_reals(ax, Cl_dnum_reals, C0_dnum_reals, fobs_gw_cents, label=None, color='deeppink')

    anis.draw_analytic(ax, Cl_redz, C0_redz, fobs_gw_cents, label='dnum, z_final', color='indigo', lw=4)
    anis.draw_reals(ax, Cl_redz_reals, C0_redz_reals, fobs_gw_cents, label=None, color='indigo')

    fig.legend(bbox_to_anchor=(0,-0.15), loc='upper left', bbox_transform = ax.transAxes, ncols=4)
    ax.set_title('Shape=%s, %s' % (str(sam.shape), str(hard_name)), fontsize=14)

    fig.tight_layout()
    return fig

## Fixed Time, small shape

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=20)  
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
fig = compare_all_analytic_anis(sam, hard, fobs_gw_cents, fobs_gw_edges)

## GW Only, small shape

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=20)  
# sam - holo.sam.Semi_Analytic_Model(mtot=(1.0e4*MSOL, 1.0e12*MSOL, 20), mrat=(1e-3, 1.0, 20), redz=(1e-3, 10.0, 20))
hard = holo.hardening.Hard_GW()
fig = compare_all_analytic_anis(sam, hard, fobs_gw_cents, fobs_gw_edges)

## Fixed Time, full shape

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=None) 
hard = holo.hardening.Fixed_Time_2PL_SAM(sam, 3*GYR)
fig = compare_all_analytic_anis(sam, hard, fobs_gw_cents, fobs_gw_edges)

## GW Only, full shape

In [ ]:
sam = holo.sam.Semi_Analytic_Model(shape=None)  
hard = holo.hardening.Hard_GW()
fig = compare_all_analytic_anis(sam, hard, fobs_gw_cents, fobs_gw_edges)

In [ ]:
print(edges[0].shape)
print(hs.shape)

In [ ]:
plt.loglog(edges[0][:-1], hs[:,0,18,5])

In [ ]:
number = holo.sam_cython.integrate_differential_number_3dx1d(edges, diff_num)
print(holo.utils.stats(number))

In [ ]:
print(np.diff(edges[1]))

In [ ]:
# integrate over log10mass
num = utils.trapz(diff_num, np.log10(edges[0]), axis=0)
# integrate over mass-ratio
num = utils.trapz(num, edges[1], axis=1)
# # integrate over redshift
# num = utils.trapz(num, edges[2], axis=2)
# # times dln(f)
# numh2 = numh2 * np.diff(np.log(fobs_gw_edges)) 
print(num.shape)
plt.loglog(edges[2][:], num[10,10,:,10])

In [ ]:
plt.loglog(edges[1][:-1], number[10,:,18,10])